<a href="https://colab.research.google.com/github/MpRonald/Deep-Learning/blob/main/PyTorch_Multiple_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [1]:
import pandas as pd
import torch.nn.functional as F
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn, optim
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

ModuleNotFoundError: No module named 'torch'

In [60]:
torch.manual_seed(123)

## Dataset

In [61]:
data = pd.read_csv('https://github.com/MpRonald/datasets/blob/main/games.csv?raw=true')
data.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
data.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

In [63]:
dataset = data[['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating']]
dataset.shape

(16719, 13)

In [64]:
dataset.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [65]:
dataset = dataset.dropna(axis=0)
dataset.isnull().sum()

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [66]:
dataset.shape

(6825, 13)

In [67]:
dataset.loc[dataset['NA_Sales'] < 1]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
85,The Sims 3,PC,2009.0,Simulation,Electronic Arts,0.99,6.42,0.00,86.0,75.0,7.6,886.0,T
94,FIFA 17,PS4,2016.0,Sports,Electronic Arts,0.66,5.75,0.08,85.0,41.0,5,398.0,E
126,FIFA 14,PS3,2013.0,Sports,Electronic Arts,0.78,4.24,0.07,86.0,37.0,4.3,576.0,E
138,World of Warcraft,PC,2004.0,Role-Playing,Activision,0.08,6.21,0.00,93.0,57.0,7.3,2182.0,T
143,FIFA 15,PS4,2014.0,Sports,Electronic Arts,0.80,4.33,0.05,82.0,47.0,5.7,988.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,E.T. The Extra-Terrestrial,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,46.0,4.0,2.4,21.0,E
16677,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,81.0,12.0,8.8,9.0,M
16696,Metal Gear Solid V: Ground Zeroes,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,80.0,20.0,7.6,412.0,M
16700,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,61.0,12.0,5.8,43.0,T


In [68]:
dataset.NA_Sales.value_counts()

0.00     543
0.07     233
0.06     228
0.04     225
0.05     223
        ... 
3.92       1
2.06       1
4.01       1
2.17       1
41.36      1
Name: NA_Sales, Length: 351, dtype: int64

In [69]:
dataset.loc[dataset['EU_Sales'] < 1]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
17,Grand Theft Auto: San Andreas,PS2,2004.0,Action,Take-Two Interactive,9.43,0.40,0.41,95.0,80.0,9,1588.0,M
48,Gran Turismo 4,PS2,2004.0,Racing,Sony Computer Entertainment,3.01,0.01,1.10,89.0,74.0,8.5,272.0,E
148,Final Fantasy XII,PS2,2006.0,Role-Playing,Square Enix,1.88,0.00,2.33,92.0,64.0,7.6,972.0,T
152,Dragon Quest IX: Sentinels of the Starry Skies,DS,2009.0,Role-Playing,Nintendo,0.63,0.67,4.35,87.0,65.0,8.8,197.0,E10+
163,Monster Hunter Freedom Unite,PSP,2008.0,Role-Playing,Capcom,0.47,0.55,4.13,81.0,48.0,8.7,124.0,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,E.T. The Extra-Terrestrial,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,46.0,4.0,2.4,21.0,E
16677,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,81.0,12.0,8.8,9.0,M
16696,Metal Gear Solid V: Ground Zeroes,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,80.0,20.0,7.6,412.0,M
16700,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,61.0,12.0,5.8,43.0,T


In [70]:
dataset.EU_Sales.value_counts()

0.00     925
0.02     644
0.01     638
0.03     474
0.04     374
        ... 
2.99       1
1.98       1
3.27       1
2.22       1
28.96      1
Name: EU_Sales, Length: 273, dtype: int64

In [71]:
dataset = dataset.loc[dataset['NA_Sales'] > 0.1]

In [72]:
dataset.NA_Sales.value_counts()

0.11    194
0.12    157
0.14    154
0.13    151
0.15    123
       ... 
2.73      1
3.13      1
3.22      1
2.30      1
2.35      1
Name: NA_Sales, Length: 340, dtype: int64

In [73]:
dataset

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936,Marvel Trading Card Game,DS,2007.0,Misc,Konami Digital Entertainment,0.11,0.00,0.00,61.0,17.0,4.9,7.0,T
9945,Onechanbara: Bikini Zombie Slayers,Wii,2008.0,Action,D3Publisher,0.11,0.00,0.00,55.0,24.0,8,20.0,M
9988,FIFA Soccer,PSP,2005.0,Sports,Electronic Arts,0.11,0.00,0.00,73.0,23.0,7.7,22.0,E
10011,Tecmo Bowl: Kickoff,DS,2008.0,Sports,Tecmo Koei,0.11,0.00,0.00,66.0,21.0,7.3,8.0,E


In [74]:
dataset = dataset.loc[dataset['EU_Sales'] > 0.1]

In [75]:
dataset.EU_Sales.value_counts()

0.12     108
0.11     106
0.15      95
0.13      85
0.20      82
        ... 
2.13       1
1.52       1
2.30       1
3.48       1
28.96      1
Name: EU_Sales, Length: 261, dtype: int64

In [76]:
dataset.shape

(2428, 13)

In [77]:
dataset['Name'].duplicated().sum()

721

In [78]:
dataset.Name.value_counts()

The LEGO Movie Videogame                                  5
Spider-Man 2                                              5
Tony Hawk's Pro Skater 3                                  5
Need for Speed: Most Wanted                               5
LEGO Marvel Super Heroes                                  5
                                                         ..
Champions of Norrath                                      1
EA Playground                                             1
Transformers                                              1
Pirates of the Caribbean: The Curse of the Black Pearl    1
SBK Superbike World Championship                          1
Name: Name, Length: 1707, dtype: int64

In [79]:
dataset = dataset.drop_duplicates(subset='Name')

In [80]:
dataset['Name'].duplicated().sum()

0

In [81]:
dataset = dataset[['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating']]

In [82]:
dataset.shape

(1707, 12)

In [83]:
dataset.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [84]:
pred = dataset.iloc[:, [0,1,2,3,7,8,9,10,11]].values
sales = dataset.iloc[:, 4:7].values

In [85]:
predict = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,2,3,8])], 
                           remainder = 'passthrough').fit_transform(pred).toarray()

In [86]:
predict.shape

(1707, 126)

In [87]:
scaler = StandardScaler()
predict = scaler.fit_transform(predict)

In [88]:
predict

array([[-0.16454729, -0.02421088, -0.20524264, ...,  0.48253194,
         0.37397831,  0.028696  ],
       [-0.16454729, -0.02421088, -0.20524264, ...,  1.43276815,
         0.61548436,  0.53091707],
       [-0.16454729, -0.02421088, -0.20524264, ...,  1.43276815,
         0.37397831, -0.14000875],
       ...,
       [-0.16454729, -0.02421088, -0.20524264, ..., -1.2019777 ,
         0.61548436, -0.38138632],
       [-0.16454729, -0.02421088, -0.20524264, ..., -0.81324471,
        -1.71907409, -0.36581357],
       [-0.16454729, -0.02421088, -0.20524264, ..., -1.41794047,
        -1.15555998, -0.37879086]])

## Building the Model

In [89]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(126,63)
        self.dense1 = nn.Linear(63,63)
        self.activation = nn.Sigmoid()
        self.output = nn.Linear(63,3)

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.output(X)
        return X

In [90]:
regressor = regressor_torch()
criterion = nn.MSELoss()
optmizer = optim.Adam(regressor.parameters())

In [91]:
predict = torch.tensor(predict, dtype=torch.float)
sales = torch.tensor(sales, dtype=torch.float)

In [92]:
dataset = torch.utils.data.TensorDataset(predict, sales)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

## Training Model

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [94]:
regressor.to(device)

regressor_torch(
  (dense0): Linear(in_features=126, out_features=63, bias=True)
  (dense1): Linear(in_features=63, out_features=63, bias=True)
  (activation): Sigmoid()
  (output): Linear(in_features=63, out_features=3, bias=True)
)

In [95]:
for epoch in range(25):
    r_loss = 0.
    r_mse0 = 0.
    r_mse1 = 0.
    r_mse2 = 0.

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optmizer.zero_grad()
        outputs = regressor.forward(inputs)

        mse_0 = F.mse_loss(outputs[:,0], labels[:,0]).item()
        mse_1 = F.mse_loss(outputs[:,1], labels[:,1]).item()
        mse_2 = F.mse_loss(outputs[:,2], labels[:,2]).item()

        r_mse0 += mse_0 * len(inputs)
        r_mse1 += mse_1 * len(inputs)
        r_mse2 += mse_2 * len(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optmizer.step()
        loss = loss.item()
        r_loss += loss * len(inputs)


        print(f"Epoch: {epoch+1} - Loop: {i+1}/{len(train_loader)} - Errors 0-> {round(mse_0,3)} --- 1-> {round(mse_1,3)} --- 2->{round(mse_2,3)}")
    
    print(f"Epoch: {epoch+1} - Errors 0-> {round(r_mse0/len(dataset),3)} --- 1-> {round(r_mse1/len(dataset),3)} --- 2->{round(mse_2/len(dataset),3)} - Cost: {round(r_loss/len(dataset),3)}")

Epoch: 1 - Loop: 1/171 - Errors 0-> 0.535 --- 1-> 0.186 --- 2->0.501
Epoch: 1 - Loop: 2/171 - Errors 0-> 0.525 --- 1-> 0.235 --- 2->0.8
Epoch: 1 - Loop: 3/171 - Errors 0-> 1.135 --- 1-> 3.947 --- 2->0.191
Epoch: 1 - Loop: 4/171 - Errors 0-> 1.205 --- 1-> 0.602 --- 2->0.762
Epoch: 1 - Loop: 5/171 - Errors 0-> 10.202 --- 1-> 2.306 --- 2->0.871
Epoch: 1 - Loop: 6/171 - Errors 0-> 2.94 --- 1-> 0.259 --- 2->0.042
Epoch: 1 - Loop: 7/171 - Errors 0-> 1.852 --- 1-> 0.391 --- 2->0.068
Epoch: 1 - Loop: 8/171 - Errors 0-> 0.224 --- 1-> 0.24 --- 2->0.028
Epoch: 1 - Loop: 9/171 - Errors 0-> 0.107 --- 1-> 0.079 --- 2->0.004
Epoch: 1 - Loop: 10/171 - Errors 0-> 0.671 --- 1-> 0.585 --- 2->0.048
Epoch: 1 - Loop: 11/171 - Errors 0-> 2.0 --- 1-> 3.107 --- 2->2.824
Epoch: 1 - Loop: 12/171 - Errors 0-> 0.397 --- 1-> 0.118 --- 2->1.663
Epoch: 1 - Loop: 13/171 - Errors 0-> 0.232 --- 1-> 0.171 --- 2->0.031
Epoch: 1 - Loop: 14/171 - Errors 0-> 0.17 --- 1-> 0.139 --- 2->0.067
Epoch: 1 - Loop: 15/171 - Errors 0-

## Evaluate Model

In [96]:
regressor.eval()

regressor_torch(
  (dense0): Linear(in_features=126, out_features=63, bias=True)
  (dense1): Linear(in_features=63, out_features=63, bias=True)
  (activation): Sigmoid()
  (output): Linear(in_features=63, out_features=3, bias=True)
)

In [100]:
predicts = predict.to(device)
pred = regressor.forward(predicts)
pred

tensor([[ 7.3532,  5.0340,  2.2306],
        [ 7.6594,  5.3162,  2.3136],
        [ 7.1749,  4.8887,  2.1670],
        ...,
        [ 0.7127,  0.3574, -0.1041],
        [ 0.3027,  0.2301,  0.1631],
        [ 0.1612,  0.1718, -0.0107]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [101]:
pred.mean(axis=0)

tensor([0.9659, 0.6848, 0.2007], device='cuda:0', grad_fn=<MeanBackward1>)

In [102]:
sales.mean(axis=0)

tensor([0.9363, 0.6386, 0.1760])